In [1]:
# Install required libraries (if not already available)
!pip3 install beautifulsoup4
!pip3 install requests

# General-purpose modules
import sys

# For sending HTTP requests
import requests

# HTML parsing
from bs4 import BeautifulSoup  # used for parsing HTML
import re
import unicodedata

# For data processing
import pandas as pd



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Extract date and time from table cells
def date_time(table_cells):
    """
    Extracts date and time information from a given HTML table cell.
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]


In [3]:
# Get booster version text from the table cell
def booster_version(table_cells):
    """
    Extracts booster version text from the given HTML table cell.
    """
    out = ''.join([booster_version for i, booster_version in enumerate(table_cells.strings) if i % 2 == 0][0:-1])
    return out


In [4]:
# Extract landing outcome from the table cell
def landing_status(table_cells):
    """
    Retrieves landing status from the given HTML cell.
    """
    out = [i for i in table_cells.strings][0]
    return out


In [5]:
# Extract payload mass from the cell
def get_mass(table_cells):
    mass = unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        new_mass = mass[0:mass.find("kg") + 2]
    else:
        new_mass = 0
    return new_mass


In [6]:
# Clean header text and extract column name
def extract_column_from_header(row):
    """
    Cleans up and returns the column name from the table header cell.
    """
    if row.br:
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
    
    column_name = ' '.join(row.contents).strip()
    if not column_name.isdigit():
        return column_name


In [7]:
# Wikipedia source URL
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Send HTTP request and parse HTML
response = requests.get(static_url)
soup = BeautifulSoup(response.text, 'html.parser')  # used for parsing HTML
